# La classification naïve bayésienne

L’algorithme de classification naïve bayésienne, largement utilisé en intelligence artificielle, repose sur le théorème de Bayes pour prédire la classe d’une donnée tout en supposant l’indépendance de ses caractéristiques.

## Hypothèse d’indépendance

### Une hypothèse bien naïve

La taille d’un chat a-t-elle une influence sur la longueur de son appendice caudal ? Le nombre d’heures de sommeil joue-t-il une quelconque importance dans l’état de vigilance d’une personne ? Le nombre d’enfants est-il en relation avec le type de la voiture qu’un foyer possède ?

À raison, on aurait tendance à répondre positivement à toutes ces questions, mais pas un classificateur naïf bayésien. Pour lui, aucune caractéristique n’influence une autre. Bien que cette hypothèse soit irréaliste dans la plupart des cas, elle a prouvé sa solidité en termes de résultats. Qui plus est, elle permet de simplifier les calculs et d’obtenir des modèles légers et rapides, ce qui en fait une méthode particulièrement efficace lorsque la quantité de données est faible ou que les ressources technologiques sont limitées.

### Application à une tâche de classification

Une tâche de classification a pour objectif d’associer à un objet une classe ($y$) en fonction de caractéristiques ($X$). On peut le traduire en termes de probabilités d’obtenir $y$ sachant $X$ :

$$
P(y \mid X) = \frac{P(X \cap y)}{P(X)} = \frac{P(X \mid y) \cdot P(y)}{P(X)}
$$

Comme $X$ est une matrice de facteurs ($x_1 \,, x_2 \,, \dots \,, x_n$) indépendants les uns des autres, la formule peut s’exprimer avec un produit de conditions indépendantes :

$$
P(y \mid x_1 \,, x_2 \,, \dots \,, x_n) = \frac{P (y) \times \prod_{i=1}^n P(x_{i} \mid y)}{\prod_{i=1}^n P(x_{i})}
$$

**Remarque :** Si les facteurs n’étaient pas jugés indépendants, la **règle du produit** qui exprime des probabilités conjointes sous forme de produits de probabilités conditionnelles entraînerait des calculs bien plus complexes :  
$$
P(x_1 \,, x_2 \,, \dots \,, x_n \mid y) = P(x_1 \mid y) \times P(x_2 \mid x_1 \cap y) \times P(x_3 \mid x_1 \cap x_2 \cap y) \times \dots \times P(x_n \mid x_1 \cap x_2 \cap \dots \cap x_{n-1} \cap, y)
$$

Et comme pour toutes les observations du jeu de données le dénominateur est constant, on peut le supprimer en inférant une notion de proportionnalité :

$$
P(y \mid x_{1} \,, x_{2} \,, \dots \,, x_n) \propto P (y) \times \prod_{i=1}^n P(x_{i} \mid y)
$$

La fonction prédictive est appliquée à l’ensemble des classes possibles et le maximum a posteriori (MAP) sert ensuite à élire la classe la plus plausible :

$$
\hat{y} = \operatorname*{argmax}_{y \in \{C_1, C_2, \dots, C_K\}} P(y) \prod_{i=1}^n P(x_i \mid y)
$$

## Les types de classificateurs

On distingue trois catégories de classificateurs :

- **Le naïf bayésien multinomial :** utilisé principalement pour les données discrètes, comme la classification de texte où l’on comptabilise les occurrences de mots.
- **Le naïf bayésien gaussien :** utilisé lorsque les données sont continues et supposent une distribution normale pour chaque caractéristique au sein de chaque classe. Si ce n’est pas le cas, une transformation peut être appliquée, voire un regroupement en classes, mais c’est au prix d’une perte sans doute conséquente d’information.
- **Le naïf bayésien de Bernoulli :** adapté aux données binaires comme lorsque l’on signale la présence ou l’absence d’un mot plutôt que de compter ses occurrences.

En pratique, il est rare qu’un jeu de données ne présente qu’un seul type de variables, aussi on adopte plutôt une approche hybride où chaque variable est modélisée conformément à sa distribution.

## Étude de cas : classification de documents

Imaginons une tâche de classification qui consiste à prédire la catégorie à laquelle un document appartient parmi trois catégories : *mystery*, *religion* ou *editorial*.

### Corpus d’apprentissage

Ci-dessous, nous avons recueilli quelques statistiques sur des textes issus du Corpus *Brown* :

| Index | TTR | >15-letter words | Quotes in foreign language? | Category |
|-|-|-|-|-|
| cl01 | 0,313816 | 1 | False | mystery   |
| cl02 | 0,283544 | 2 | False | mystery   |
| cl03 | 0,303542 | 0 | True  | mystery   |
| cd01 | 0,335743 | 1 | True  | religion  |
| cd02 | 0,314910 | 9 | False | religion  |
| cd03 | 0,357204 | 3 | True  | religion  |
| cb01 | 0,434545 | 3 | False | editorial |
| cb02 | 0,410027 | 6 | False | editorial |
| cb03 | 0,404635 | 2 | False | editorial |

### Description des variables

Le premier réflexe face à un jeu de données est de décrire les variables qui le constituent :

- **TTR** : *Type Token Ratio*, indicateur de diversité lexicale qui prend ses valeurs sur une échelle de $[0,1]$.
- **>15-letter words :** Nombre de mots contenant plus de 15 lettres.
- **Quotes in foreign language :** Le texte contient-il des citations en langue étrangère ?
- **Category :** Catégorie du document.

**Remarque :** Les données de la variable *Quotes in foreign language?* sont inventées.

La variable *Category* est la **variable cible** (ou *target* en anglais). Dans le contexte d’une tâche de classification, elle consigne la classe du document. Toutes les autres variables, à l’exception de la variable *Index* que nous écartons, sont dites **explicatives** au sens où elles servent à expliquer l’attribution de la classe au document.

Il est de bon ton ensuite d’explorer les données afin de vérifier leur intégrité (données manquantes ou aberrantes) puis de calculer quelques mesures de tendance centrale (moyenne, médiane…) et de dispersion (variance, écart-type…), le tout accompagné de diagrammes, afin d’en avoir une bonne vision d’ensemble. Nous en faisons ici l’économie pour nous concentrer sur la méthode pour établir correctement notre modèle de classification.

### Document à classer

Soit un nouveau document auquel nous souhaiterions attribuer l’étiquette la plus plausible en fonction des indicateurs suivants :

- **TTR :** 0,410961 ;
- **>15-letter words :** 4 ;
- **Quotes in foreign language :** *True*.

### Modéliser une variable binaire

Attachons-nous en premier lieu à la variable *Quotes in foreign language*. Comme il s’agit d’une variable qui ne peut prendre que deux valeurs (*True* ou *False*), elle est dite **binaire** et, pour calculer la probabilité que notre document contienne de telles citations en langue étrangère, nous décidons de la modéliser comme si elle suivait une loi de Bernoulli où la probabilité conditionnelle dépend uniquement du nombre de réalisations de l’événement dans chaque classe :

$$
\begin{aligned}
  P(\text{Quotes in foreign language} \mid \text{mystery}) &= \dfrac{1}{3}\\
  P(\text{Quotes in foreign language} \mid \text{religion}) &= \dfrac{2}{3}\\
  P(\text{Quotes in foreign language} \mid \text{editorial}) &= \dfrac{0}{3}
\end{aligned}
$$

**Remarque :** Du nom du mathématicien suisse Jacques Bernoulli, la loi de Bernoulli modélise une variable aléatoire qui ne peut prendre que deux valeurs : 1 (succès) avec une probabilité $p$, et 0 (échec) avec une probabilité $1−p$.

### Modéliser une variable continue

La variable *TTR* contient des données dites continues, dans le sens où elles peuvent prendre n’importe quelle valeur réelle sur un intervalle. Cette liberté empêche d’effectuer des calculs de fréquences, il serait en effet absurde de compter le nombre de fois où le *TTR* vaut 0,303542 ou 0,404635 ou encore 0,004569…

S’il nous apparaît évident que la variable *TTR* ne suit pas une loi de Bernoulli comme dans le cas de la variable *Quotes in foreign language*, quelle loi choisir alors pour la modéliser ? Nous décidons arbitrairement de la traiter comme si elle suivait une loi normale et utilisons une modélisation gaussienne de la vraisemblance $P(x_{i} \mid y)$, sous forme de fonction de densité :

$$
P(x_{i} \mid y) \propto \frac{1}{\sqrt{2 \pi \sigma^{2}_{y}}} \exp \left(- \frac{\left(x_{i} - \mu_{y}\right)^{2}}{2\sigma^{2}_{y}} \right)
$$

Où :

- $x_i$ prend la valeur de la variable $x$ pour une observation donnée ;
- $\mu_y$ représente l’espérance de la variable conditionnelle à $y$ (moyenne arithmétique des observations associées à la classe $y$) ;
- $\sigma^2_y$ représente la variance non biaisée de l’échantillon en mesurant la dispersion des valeurs de $y$ autour de la moyenne.

**Remarque :** Si dans la réalité nous devrions vérifier d’abord l’hypothèse de normalité de la variable avant de l’exploiter, nous supposons ici à des fins pédagogiques que les données forment un échantillon d’une distribution gaussienne. Dans le cas contraire, nous aurions eu le choix entre deux alternatives : soit la transformer, avec par exemple un logarithme, soit effectuer des regroupements pour en faire une variable catégorielle. Notons en dernier lieu que le *Type Token Ratio* a très peu de chances de suivre une loi normale.

#### Calculer l’espérance mathématique

Nous évaluons l’espérance de la variable *TTR* à la moyenne de ses valeurs :

$$
\bar{y} = \frac{1}{n} \sum_{i=1}^{n} y_i
$$

Ce qui donne, pour chaque catégorie :

$$
\begin{aligned}
\bar{y}_\text{mystery} &= \frac{1}{3} \cdot 0,900902 = 0,3003\\
\bar{y}_\text{religion} &= \frac{1}{3} \cdot 1,007857 = 0,33595\\
\bar{y}_\text{editorial} &= \frac{1}{3} \cdot 1,249207 = 0,4164
\end{aligned}
$$

#### Calculer la variance

La variance se définit comme l’espérance des écarts d’une variable $X$ à la moyenne réelle de la population. Dans le cas où l’on calcule la variance d’un échantillon, il est de coutume d’appliquer un degré de liberté de 1 afin d’obtenir un estimateur non biaisé :

$$
\sigma^2 = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \bar{x})^2
$$

La variance de la variable *TTR* vaut ainsi, pour chaque catégorie :

$$
\begin{aligned}
\sigma^2_\text{mystery} &= \frac{1}{3-1} \cdot ((0,313816 - 0,3003)^2 + (0,283544 - 0,3003)^2 + (0,303542 - 0,3003)^2)\\
&= \frac{1}{2} \cdot (0,00018268 + 0,00028076 + 0,00001051)\\
&= 0,00023698\\
\sigma^2_\text{religion} &= \frac{1}{3-1} \cdot ((0,335743 - 0,33595)^2 + (0,314910 - 0,33595)^2 + (0,357204 - 0,33595)^2)\\
&= \frac{1}{2} \cdot (0,00000004 + 0,00044268 + 0,00045173)\\
&= 0,00044722\\
\sigma^2_\text{editorial} &= \frac{1}{3-1} \cdot ((0,434545 - 0,4164)^2 + (0,410027 - 0,4164)^2 + (0,404635 - 0,4164)^2)\\
&= \frac{1}{2} \cdot (0,00032924 + 0,00004062 + 0,00013842)\\
&= 0,00025414
\end{aligned}
$$

#### Estimer la probabilité

En remplaçant les termes de la fonction de densité de probabilité, nous obtenons :

$$
\begin{aligned}
P(\text{TTR = 0,410961} \mid \text{mystery}) &\propto \frac{1}{\sqrt{2 \cdot \pi \cdot 0,00023698}} \exp \left(- \frac{\left(0,410961 - 0,3003\right)^{2}}{2 \cdot 0,00023698} \right)\\
&\propto 1,55 \times 10^{-10}\\
P(\text{TTR = 0,410961} \mid \text{religion}) &\propto \frac{1}{\sqrt{2 \cdot \pi \cdot 0,00044722}} \exp \left(- \frac{\left(0,410961 - 0,33595\right)^{2}}{2 \cdot 0,00044722} \right)\\
&\propto 0,0349\\
P(\text{TTR = 0,410961} \mid \text{editorial}) &\propto \frac{1}{\sqrt{2 \cdot \pi \cdot 0,00025414}} \exp \left(- \frac{\left(0,410961 - 0,4164\right)^{2}}{2 \cdot 0,00025414} \right)\\
&\propto 23,62
\end{aligned}
$$

#### Interprétation

La densité de probabilité associée à un *TTR* de 0,410961 pour la catégorie *mystery* est très faible, un résultat qui s’explique par un écart plutôt important de la moyenne alors même que l’écart-type (la racine carrée de la variance) est lui-même minime.

Pour la catégorie *religion*, bien que le *TTR* soit également éloigné de la moyenne, un écart-type plus grand permet d’en atténuer l’impact, ce qui se traduit par une densité de probabilité plus élevée.

Enfin, la densité maximale est obtenue pour la catégorie *editorial* : le *TTR* du nouveau document est très proche de la moyenne de cette classe alors même que nous savons sa variance marginale.

Si nous traçons les courbes de densité par classe, nous observons bien que la densité pour la classe *editorial* est très élevée alors que les autres sont proches de 0 :

![Un intervalle de ± 0,001 autour du TTR observé (0,410961) croise les courbes de densité pour les trois classes à des hauteurs différentes. La densité pour la classe *editorial* est la plus élevée des trois.](./figs/TTR_densities.png)

Affichage des courbes de densité par classes. La densité pour le TTR observé (0,410961) est la plus élevée pour la classe 'editorial'.

Le résultat obtenu est une densité et non une probabilité qui, elle, pour rappel, s’exprime dans un intervalle $[0,1]$. Pour obtenir une probabilité, nous pouvons utiliser une fonction de répartition d’une loi normale (CDF pour *Cumulative Distribution Function*) ou simplement consulter la table de la loi normale.

Par exemple, pour estimer la probabilité qu’un *TTR* pour la classe *editorial* tombe dans un intervalle autour de 0.410961 ± 0.001 sachant que $\mu_{\text{editorial}} = 0.4164$ et $\sigma_{\text{editorial}} = \sqrt{0.00025414} \approx 0.01593$ nous posons :

$$
\begin{aligned}
  P(0.410961 - 0.001 < TTR < 0.410961 + 0.001) &\approx \Phi(−0,278) − \Phi(−0,404) \\
  &\approx 0.3903 − 0.3438 \\
  &= 0.0465
\end{aligned}
$$

De là, la probabilité de tomber sur un TTR à ± 0.001 autour de 0.410961 est de 4,65 % pour la classe *editorial*.

### Modéliser une variable discrète

La variable *>15-letter words* est représentée par des valeurs discrètes qui prennent l’apparence de nombres entiers et strictement positifs. Comme en plus elle rend compte d’un événement rare, les mots de plus de quinze lettres dans un texte anglais étant peu fréquents, on a tout intérêt à la traiter comme une distribution de Poisson (asymétrie à droite car il est plus fréquent de rencontrer très peu de mots de plus de quinze lettres).

![La distribution des mots de plus de quinze lettres dans le corpus Brown montre une asymétrie à droite caractéristique d’une loi de Poisson.](./figs/words_distribution.png)

La distribution des mots de plus de quinze lettres dans le corpus Brown montre une asymétrie à droite caractéristique d’une loi de Poisson.

Dans ce cas, nous considérons la fonction de masse de probabilité de Poisson :

$$
P(X = k) = \frac{\lambda^k \cdot e^{-\lambda}}{k!}
$$

Où :

- $k$ est le nombre de mots de plus de 15 lettres observé ;
- $\lambda$ est le taux moyen d’occurrence observé pour la catégorie.

Calculons d’abord $\lambda$ pour chaque catégorie :

$$
\begin{aligned}
\lambda_{mystery} &= \frac{1 + 2 + 0}{3} = 1\\
\lambda_{religion} &= \frac{1 + 9 + 3}{3} = 4,33\\
\lambda_{editorial} &= \frac{3 + 6 + 2}{3} = 3,67
\end{aligned}
$$

À présent, calculons la probabilité que notre document appartienne à une catégorie particulière :

$$
\begin{aligned}
P(\text{>15-letter words = 4} \mid \text{mystery}) &= \frac{1^4 \cdot e^{-1}}{4!}\\
&= \frac{1 \cdot 0,3679}{24}\\
&= 0,0153\\
P(\text{>15-letter words = 4} \mid \text{religion}) &= \frac{4,33^4 \cdot e^{-4,33}}{4!}\\
&= \frac{351,52125121 \cdot 0,013}{24}\\
&\approx 0,1904\\
P(\text{>15-letter words = 4} \mid \text{editorial}) &= \frac{3,67^4 \cdot e^{-3,67}}{4!}\\
&= \frac{181,41126721 \cdot 0,025}{24}\\
&\approx 0,189
\end{aligned}
$$

### Modèle de classification

Maintenant que nous connaissons toutes les probabilités pour chaque variable, nous pouvons calculer la vraisemblance selon laquelle notre document appartienne à une catégorie ou l’autre. En nous fondant sur le théorème de Bayes, nous pouvons établir le modèle suivant :

$$
P(y \mid X) \propto P(y) \cdot P(X_{\text{TTR}} \mid y) \cdot P(X_{\text{>15-letter words}} \mid y) \cdot P(X_{\text{Quotes in foreign language}} \mid y)
$$

Et puisque le prior $P(y)$ est le même pour les trois classes ($\frac{1}{3}$), la vraisemblance $P(X \mid y)$ suffit à elle seule pour le calcul de la probabilité a posteriori, à un facteur de normalisation près. Pour nos trois classes, nous obtenons les scores suivants :

$$
\begin{aligned}
P(\text{mystery} \mid X) &\propto 1,55 \times 10^{-10} \cdot 0,0153 \cdot \frac{1}{3}\\
&\propto 7,905 \times 10^{-13}\\
P(\text{religion} \mid X) &\propto 0,0349 \cdot 0,1904 \cdot \frac{2}{3}\\
&\propto 0,0044\\
P(\text{editorial} \mid X) &\propto 23,62 \cdot 0,189 \cdot \frac{0}{3}\\
&\propto 0
\end{aligned}
$$

À ce stade, nous pouvons conclure que :

$$
P(\text{religion} \mid X) > P(\text{mystery} \mid X) > P(\text{editorial} \mid X)
$$

#### Lissage de Laplace

Un problème classique avec la classification naïve bayésienne est que si l’une des probabilités conditionnelles (ou l’un des scores de vraisemblance comme dans notre cas) est égale à 0, alors tout le produit $P(X \mid y_i)$ est annulé et ce même si les autres termes sont significatifs.

Pour éviter ce genre d’écueil, il est courant d’utiliser une méthode de lissage qui consiste à ajouter une petite valeur à chaque donnée. L’une de ces méthodes s’appelle **lissage de Laplace** : on rajoute un paramètre $\alpha$ au numérateur que l’on ajoute également au dénominateur mais en le multipliant par le nombre $k$ de valeurs possibles. Dans le cas de la variable *Quotes in foreign language?*, en prenant $\alpha = 1$ cela donne :

$$
\begin{aligned}
P(\text{mystery} \mid X) &\propto 1.55 \times 10^{-10} \cdot 0.0153 \cdot \frac{1 + 1}{3 + 1 \cdot 2}\\
&\propto 1.55 \times 10^{-10} \cdot 0.0153 \cdot \frac{2}{5}\\
&\propto 9.486 \times 10^{-13}\\
P(\text{religion} \mid X) &\propto 0.0349 \cdot 0.1904 \cdot \frac{2 + 1}{3 + 1 \cdot 2}\\
&\propto 0.0349 \cdot 0.1904 \cdot \frac{3}{5}\\
&\propto 0.004\\
P(\text{editorial} \mid X) &\propto 23.62 \cdot 0.189 \cdot \frac{0 + 1}{3 + 1 \cdot 2}\\
&\propto 23.62 \cdot 0.189 \cdot \frac{1}{5}\\
&\propto 0.893
\end{aligned}
$$

Nous pouvons conclure que, après application du lissage de Laplace, la classe la plus plausible pour le document dont les paramètres sont $[0,410961, 4, \text{True}]$ est la classe *editorial* avec un score de 0,893.

#### Normalisation des scores

Le score obtenu n’est pas à proprement parler une probabilité puisqu’il mélange une densité (pour la variable *TTR*) et des probabilités. Cela ne pose toutefois pas de problème comme notre modèle adopte une **approche naïve bayésienne mixte** qui assume l’idée d’obtenir des scores non normalisés afin de ne sélectionner que le plus élevé.

Si nous souhaitons normaliser les scores, nous n’avons plus qu’à appliquer le théorème de Bayes. Comme nous connaissons $P(X \mid \text{editorial})$, nous pouvons facilement calculer $P(\text{editorial} \mid X)$ :

$$
\begin{aligned}
P(\text{editorial} \mid X) &= \frac{P(X \mid \text{editorial}) \cdot P(\text{editorial})}{P(X)}\\
&= \frac{0.893 \cdot 0.33}{P(X)}
\end{aligned}
$$

Pour obtenir $P(X)$, nous utilisons la formule des probabilités totales :

$$
\begin{aligned}
P(X) &= P(X \mid \text{editorial}) \cdot P(\text{editorial}) + P(X \mid \text{mystery}) \cdot P(\text{mystery}) + P(X \mid \text{religion}) \cdot P(\text{religion})\\
&= 0.893 \cdot 0.33 + 0.004 \cdot 0.33 + 9.486 \times 10^{-13} \cdot 0.33\\
&= 0.295 + 0.00132 + 3.13 \times 10^{-13}\\
&= 0.296
\end{aligned}
$$

De là, nous pouvons résoudre :

$$
\begin{aligned}
P(\text{mystery} \mid X) &= \frac{9.486 \times 10^{-13} \cdot 0.33}{0.296} = 1.057 \times 10^{-12}\\
P(\text{religion} \mid X) &= \frac{0.004 \cdot 0.33}{0.296} = 0.0045\\
P(\text{editorial} \mid X) &= \frac{0.893 \cdot 0.33}{0.296} = 0.9966
\end{aligned}
$$

Au final, nous pouvons conclure qu’il y a 99,66 % de chances que le nouveau document à classer appartienne à la catégorie *editorial*.